In [7]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import random
import numpy as np

class TicTacToe:
    def __init__(self):
        self.board = np.zeros((3, 3), dtype=int)  # 0=empty, 1=X (Human), -1=O (AI)
        self.current_player = 1  # Human starts
        self.difficulty = "medium"  # easy, medium, hard
        self.buttons = [[None]*3 for _ in range(3)]
        self.output = widgets.Output()
        self.reset_button = widgets.Button(description='Reset Game')
        self.reset_button.on_click(self.reset_game)

        self.create_difficulty_selector()
        self.create_board()
        self.display_board()

    def create_difficulty_selector(self):
        self.diff_var = widgets.Dropdown(
            options=['easy', 'medium', 'hard'],
            value='medium',
            description='Difficulty:',
        )
        self.diff_var.observe(self.change_difficulty, names='value')

    def create_board(self):
        for i in range(3):
            for j in range(3):
                button = widgets.Button(description='', layout=widgets.Layout(width='60px', height='60px'))
                button.on_click(lambda b, i=i, j=j: self.human_move(i, j))
                self.buttons[i][j] = button

    def display_board(self):
        clear_output()
        display(self.diff_var)
        display(widgets.VBox([widgets.HBox(self.buttons[0]),
                               widgets.HBox(self.buttons[1]),
                               widgets.HBox(self.buttons[2]),
                               self.reset_button,
                               self.output]))

    def human_move(self, i, j):
        if self.board[i][j] == 0 and self.current_player == 1:
            self.board[i][j] = 1  # Human = X
            self.buttons[i][j].description = 'X'
            self.buttons[i][j].disabled = True

            if not self.check_winner():
                self.current_player = -1  # AI's turn
                self.ai_move()

    def ai_move(self):
        empty_spots = [(i, j) for i in range(3) for j in range(3) if self.board[i][j] == 0]

        if self.difficulty == "easy":
            # Random moves (human can win)
            i, j = random.choice(empty_spots)
        elif self.difficulty == "medium":
            # Smarter AI (blocks wins but not perfect)
            move = self.find_winning_move(-1)
            if not move:
                move = self.find_winning_move(1)
            if not move:
                if (1, 1) in empty_spots:
                    move = (1, 1)
                else:
                    move = random.choice(empty_spots)
            i, j = move
        else:
            # Hard AI (Minimax - unbeatable)
            best_move = self.minimax(self.board, -1)["position"]
            i, j = best_move

        self.board[i][j] = -1  # AI = O
        self.buttons[i][j].description = 'O'
        self.buttons[i][j].disabled = True
        self.check_winner()
        self.current_player = 1  # Human's turn again

    def find_winning_move(self, player):
        for i, j in [(i, j) for i in range(3) for j in range(3) if self.board[i][j] == 0]:
            self.board[i][j] = player
            if self.check_for_winner():
                self.board[i][j] = 0  # Undo move
                return (i, j)
            self.board[i][j] = 0
        return None

    def minimax(self, board, player):
        max_player = -1  # AI
        min_player = 1  # Human

        # Check if game is over
        if self.check_for_winner(min_player):
            return {"score": -1, "position": None}
        elif self.check_for_winner(max_player):
            return {"score": 1, "position": None}
        elif len([(i, j) for i in range(3) for j in range(3) if board[i][j] == 0]) == 0:
            return {"score": 0, "position": None}

        if player == max_player:
            best = {"score": -np.inf, "position": None}
            for i, j in [(i, j) for i in range(3) for j in range(3) if board[i][j] == 0]:
                board[i][j] = player
                sim_score = self.minimax(board, min_player)
                board[i][j] = 0  # Undo
                sim_score["position"] = (i, j)
                if sim_score["score"] > best["score"]:
                    best = sim_score
        else:
            best = {"score": np.inf, "position": None}
            for i, j in [(i, j) for i in range(3) for j in range(3) if board[i][j] == 0]:
                board[i][j] = player
                sim_score = self.minimax(board, max_player)
                board[i][j] = 0  # Undo
                sim_score["position"] = (i, j)
                if sim_score["score"] < best["score"]:
                    best = sim_score
        return best

    def check_winner(self):
        winner = None
        if self.check_for_winner(1):
            winner = "Human"
        elif self.check_for_winner(-1):
            winner = "AI"
        elif all(self.board[i][j] != 0 for i in range(3) for j in range(3)):
            winner = "Draw"

        if winner:
            msg = f"{winner} wins!" if winner != "Draw" else "It's a draw!"
            with self.output:
                clear_output(wait=True)
                print(msg)
            self.reset_game()
            return True
        return False

    def check_for_winner(self):
        # Check rows, columns, and diagonals
        for i in range(3):
            if all(self.board[i][j] == 1 for j in range(3)) or \
               all(self.board[j][i] == 1 for j in range(3)):
                return True
        if (self.board[0][0] == self.board[1][1] == self.board[2][2] == 1) or \
           (self.board[0][2] == self.board[1][1] == self.board[2][0] == 1):
            return True
        return False

    def change_difficulty(self, change):
        self.difficulty = self.diff_var.value
        self.reset_game()

    def reset_game(self, b=None):
        self.board = np.zeros((3, 3), dtype=int)
        self.current_player = 1
        for i in range(3):
            for j in range(3):
                self.buttons[i][j].description = ''
                self.buttons[i][j].disabled = False
        with self.output:
            clear_output(wait=True)

# To run the game in Colab
game = TicTacToe()


Dropdown(description='Difficulty:', index=1, options=('easy', 'medium', 'hard'), value='medium')